## 기존 Pose Estimate 수행

### 웹캠으로 영상을 받아와 Pose Estimate 수행

어플 상 화면 : 카메라가 받은 FPS로 그대로 출력

Pose Estimate : FPS 2 단위로 알로리즘 추출 후 특정 자세 추출시 촬영 수행

In [11]:
import cv2
import numpy as np
import time

In [26]:
# fashion_pose.py : MPII를 사용한 신체부위 검출


# MPII에서 각 파트 번호, 선으로 연결될 POSE_PAIRS
BODY_PARTS = { "Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
                ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
                ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
                ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]
    
# 각 파일 path
protoFile = "pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose/mpi/pose_iter_160000.caffemodel"

 
# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)


In [27]:
def pose_estimate(frame):

    # frame.shape = 불러온 이미지에서 height, width, color 받아옴
    inWidth = 368
    inHeight = 368
    frameHeight, frameWidth, _ = frame.shape

    # network에 넣기위해 전처리
    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)

    # network에 넣어주기
    net.setInput(inpBlob)

    # 결과 받아오기
    output = net.forward()

    # output.shape[0] = 이미지 ID, [1] = 출력 맵의 높이, [2] = 너비
    H = output.shape[2]
    W = output.shape[3]

    # 키포인트 검출시 이미지에 그려줌
    points = []
    for i in range(0,15):
        # 해당 신체부위 신뢰도 얻음.
        probMap = output[0, i, :, :]

        # global 최대값 찾기
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        # 원래 이미지에 맞게 점 위치 변경
        x = (frameWidth * point[0]) / W
        y = (frameHeight * point[1]) / H

        # 키포인트 검출한 결과가 0.1보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로    
        if prob > 0.1 :    
            cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)       # circle(그릴곳, 원의 중심, 반지름, 색)
            cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, lineType=cv2.LINE_AA)
            points.append((int(x), int(y)))
        else :
            points.append(None)

    # 이미지 복사
    frameCopy = frame

    # 각 POSE_PAIRS별로 선 그어줌 (머리 - 목, 목 - 왼쪽어깨, ...)
    for pair in POSE_PAIRS:
        partA = pair[0]             # Head
        partA = BODY_PARTS[partA]   # 0
        partB = pair[1]             # Neck
        partB = BODY_PARTS[partB]   # 1

        #print(partA," 와 ", partB, " 연결\n")
        if points[partA] and points[partB]:
            cv2.line(frameCopy, points[partA], points[partB], (0, 255, 0), 2)
            
    return frameCopy


In [28]:
# 카메라를 구동하여 흑백으로 표출해주는 함수

def showVideo(FPS):

    try:
        print('카메라를 구동합니다.')
        cap = cv2.VideoCapture(0) # VideoCapture 객체 생성 -> 구동할 장치 번호 입력(캠이 하나임으로 0번)
    except:                      # 만약 캠구동이 아니라 저장된 파일을 재생하기 원할 시, 경로와 파일이름 입력
        print('카메라 구동 실패')
        return

    cap.set(3, 480) # 프레임 크기 설정
    cap.set(4, 320)    
    prev_time = 0
    
    while True:
        ret, frame = cap.read() # 재생되는 비디오를 한 프레임씩 읽기 
        frame = cv2.flip(frame, 1) # 제대로 읽을 시에 ret : True or False, frame : 읽은 프레임
        current_time = time.time() - prev_time
        
        if not ret:
            print('비디오 읽기 오류')
            break
        
        framePose = pose_estimate(frame)
        
        if (ret is True) and (current_time > 1./ FPS) :
            prev_time = time.time()
            cv2.imshow('video', framePose) # 변환한 프레임을 화면에 디스플레이
        
        
        k = cv2.waitKey(1) & 0xFF
        if k == 27:
            print('카메라 구동을 종료합니다.')
            break
    
    cap.release() # 오픈한 cap 객체 해제 **** 필수 ****
    cv2.destroyAllWindows() # 윈도우 창 답기

In [33]:
showVideo(2)

카메라를 구동합니다.
카메라 구동을 종료합니다.


In [31]:
start = time.time()
frame = cv2.imread('images/single.jpg')
frameOut = pose_estimate(frame)
cv2.imshow("Output-Keypoints",frameOut)
cv2.waitKey(0)
cv2.destroyAllWindows()
print('time :', time.time() - start)

time : 0.4129946231842041


### 프레임 조절

In [4]:
input_source = "../2.데이터/0.dataset/test.mp4"
cap = cv2.VideoCapture(input_source)

while True:
    ret, frame = cap.read() # 재생되는 비디오를 한 프레임씩 읽기 
    frame = cv2.flip(frame, 1) # 제대로 읽을 시에 ret : True or False, frame : 읽은 프레임

    if not ret:
        print('비디오 읽기 오류')
        break

    framePose = pose_estimate(frame)

    cv2.imshow('video', framePose) # 변환한 프레임을 화면에 디스플레이
    
    k = cv2.waitKey(1)
    if k == 27: break
